### Problema
**El Triángulo**  
Javier estaba un día practicando con su instrumento favorito: el triángulo. El triángulo es un instrumento musical tan espectacular que sus notas se escriben como números enteros. Este día Javier se propuso componer una canción de una forma bastante peculiar. Tomó $n$ enteros (notas) aleatorios y los escribió en una lista $a$. Una melodía válida es una subsecuencia de $a$ en la que todos sus números adyacentes cumplen que:
* Se diferencian en 1.
* Son congruentes módulo 7.

La canción de Javier debe contener exactamente 4 melodías que cumplan con lo anterior y además no intercepten entre sí. Ayude a Javier encontrando una canción que maximice las notas usadas.


Se asume que:
* La secuencia de notas tiene longitud mayor o igual que 4.
* La solución implementada devuelve la cantidad máxima de notas en la canción.

### Modelación y Correctitud
Con el objetivo de maximizar las subsecuencias que se pueden extraer de $a$ sin que se intercepten, se debe tener en cuenta crear un grafo dirigido $G$ que respete este criterio, por lo que todos los valores $a_i$ son vértices de $G$ ya que cada uno por si solo es una subsecuencia de $a$.
Luego cada uno de estos vértices podría formar una subsecuencia con otros vértices cuyo valor este en una posición superior en el array y cumpla los requisitos expuestos en el problema, finalmente:
* $G$ grafo del sistema
* $\forall i,j ~\in~ a$: $~~~i \rightarrow j \in A(G)~~$ si $~~i < j~~$ y se cumple que: $~|a_i - a_j| = 1~~$ o $~~(a_i - a_j)\%7 = 0$
* $cap(u,v)$ función de capacidad en la arista
* $cost(u,v)$ se agrega la función del costo para tomar en cuenta esta arista en la secuencia

Para la resolución de este problema se realizará un algoritmo de flujo máximo (en este caso 4) con costo mínimo.  
Redefinamos el grafo original de la siguiente forma:
* $G_f = <V,A>$ una red de flujo
* $s$: fuente $\in V(G_f)$
* $t$: receptor $\in V(G_f)~$ al cual solo llega flujo máximo 4, que son las 4 subsecuencias que se desea encontrar
* $\forall v_i \in V(G)$ se divide en $v_{i1} \in G_f$ y $v_{i2} \in G_f$ y se adiciona una arista dirigida $v_{i1} \rightarrow v_{i2} \in A(G_f)$ donde $~cap(v_{i1},v_{i2}) = 1~$ y $~cost(v_{i1},v_{i2}) = -1~$ que representa la decisión de tomar este elemento como parte de la subsecuencia a formar
* $\forall v_i \in V(G)~$ se tiene $~s \rightarrow v_{i1} \in A(G_f)~$ ya que, como se mencionó anteriormente, cada elemento por si solo puede ser una subsecuencia o el punto de partida de una, donde $~cap(s,v_{i1}) = 1~$ y $~cost(s,v_{i1}) = 0$
* $\forall i,j~~$ si $v_i \rightarrow v_j \in A(G)$ entonces $~v_{i2} \rightarrow v_{j1} \in A(G_f)~$ con $~cap(v_{i2},v_{j1}) = 1~$ y $~cost(v_{i2},v_{j1}) = 0$
* $\forall v_i \in V(G)~$ se tiene $~v_{i2} \rightarrow t \in A(G_f)~$ para terminar la subsecuencia, donde $~cap(v_{i2},t) = 1~$ y $~cost(v_{i2},t) = 0$

Luego, en la red residual para cada cada arista de retroceso $~u \rightarrow v$ se cumple que $cap_{0}(v,u) = 0 $ y $cost(v,u) =  ~-cost(u,v)$ 

La idea tras la modelación anterior esta asociada, a que cada vez que se utiliza una nota en un camino aumentativo se disminuye el costo de dicho camino, por lo que el algoritmo trataría en todo momento de maximizar el flujo a 4 con el menor costo posible (es decir con la mayor cantidad de notas). 

La diferencia de este algoritmo con el algoritmo de flujo máximo clásico, es que cuando se computa un camino aumentativo, en lugar de buscar el camino mas corto hacia el destino (usando BFS), se busca el camino de costo mínimo hacia el destino (usando el algoritmo de Bellman-Ford en este caso, dado que existen aristas con costo negativo).

El grafo original es un DAG, dado que solo se tienen aristas en una sola dirección (hacia delante) en la lista de notas. Luego en la red residual se forman ciclos con la introducción de las aristas de retroceso, pero el costo del ciclo formado por una arista del grafo original y su arista de retroceso es cero ($cost(v,u) =  ~-cost(u,v)$). Por tanto, es posible utilizar Bellman-Ford para encontrar un camino de costo mínimo en la red residual, dado que se garantiza que no existen ciclos de costo negativo en dicha red.


### Solución
Una posible implementación del algoritmo es la siguiente:

Creación del grafo Explicaciooooooooooooooooooooooooooooooooooooooooooooooooonnnnnnnnnnnnnnnnnnnnnnnnnnn

In [ ]:
from collections import defaultdict as dd
from math import inf
class edge:
    def __init__(self, cap:int, cost:int):
        self.cap = cap
        self.cost = cost

def add_edge(graph:dict[int, dict[int, edge]], v:int, u:int, cap:int, cost:int):
    graph[v][u] = edge(cap,  cost) # arista del grafo v -> u
    graph[u][v] = edge(  0, -cost) # arista inversa del grafo residual u -> v

def add_edge_list(graph:dict[int, dict[int, edge]], v:int, index:list[int]):
    for elem in index: add_edge(graph, v, elem, 1, 0)

def create_residual_graph(a:list[int]):
    graph:dict[int, dict[int, edge]] = dd(lambda:dd(lambda:None))  # grafo residual donde por cada nota hay 2 vertices y cada arista tiene su inversa
    n = len(a)
    s = 0
    t = 2*n+1
    d = 2*n+2
    refer:dict[int, list[int]] = dd(list)          
    modul:dict[int, list[int]] = dd(list)        

    for i in range(n, 0, -1):
        value = a[i-1]
        add_edge(graph, s, 2*i-1, 1, 0)       # este es vi,1
        add_edge(graph, 2*i-1, 2*i, 1, -1)    # este es vi,2
        add_edge(graph, 2*i, t, 1, 0)     
          
        ig = refer[value+1]
        il = refer[value-1]
        im = modul[value%7]
        
        if len(ig) > 0: add_edge_list(graph, 2*i, ig)
        if len(il) > 0: add_edge_list(graph, 2*i, il)
        if len(im) > 0: add_edge_list(graph, 2*i, im)
        refer[value].append(2*i-1)
        modul[value%7].append(2*i-1)     
    add_edge(graph, t, d, 4, 0)

    return graph, s, d

Algoritmo Bellman-Ford

La complejidad computacional de este algoritmo es $O(|V||E|)$ 

In [ ]:
def Bellman_Ford(G:dict[int, dict[int, edge]], s:int):
    d = dd(lambda:inf)
    d[s] = 0
    pi = dd(lambda:None)
    for _ in range(len(G.keys())-1):
        for v1,edges in G.items():
            for v2,p in edges.items():
                if not p==None and not p.cap == 0 and not d[v1]==inf and d[v2]> d[v1]+ p.cost:
                    d[v2]=d[v1]+p.cost
                    pi[v2]=v1
    for v1,edges in G.items():
        for v2,p in edges.items():
            if not p==None and not p.cap == 0 and not d[v2]==inf and not d[v1]==inf and  d[v2]>d[v1]+p.cost:
                return None
    return pi

Algoritmo MinCost-MaxFlow


El costo de esta implementación es igual la cantidad de veces que se ejecuta el ciclo while multiplicado por el costo del algoritmo find_path, adicionado a C(create_residual_graph)+O(n^{2}), donde $n$ es  $n$ la cantidad de vértices del grafo original.

En cada iteración del ciclo while el flujo que se pasa por la red original aumenta en la capacidad del camino aumentativo en cuestión, que es como mínimo 1. Luego la cantida máxima de veces que se ejecuta este ciclo es igual al valor del flujo máximo, en este caso un valor constante menor que 4.

El algoritmo find_path computa el camino de costo mínimo hasta el destino en la red residual ($m^{2}n$) y luego reconstruye dicho camino (O(m^{2})), por lo cual el costo es O($m^{2}n$), donde $m$ es la cantidad de aristas del grafo original y $n$ la cantidad de vértices.

Finalmente la complejidad del algoritmo es O($m^{2}n$)

In [ ]:
def min_cost_flow(a:list[int]):
    G_f, s, t = create_residual_graph(a)
    output = 0
    flow = dd(lambda:dd(lambda:None))

    for v1,edges in G_f.items():
        for v2,_ in edges.items():
            if v2 > v1: flow[v1][v2]=0
    
    path, cap = find_path(G_f, s, t)   
    while not len(path)==0:
        for v1, v2 in path:
            if  v2 < v1: # la arista no esta en la red original por tanto es una arista de retroceso
                if G_f[v1][v2].cost == -1: output-=1
                flow[v2][v1] -= cap  #actualizar red original 
                G_f[v1][v2].cap = G_f[v1][v2].cap - cap   #actualizar arista de retroceso en la red residual
                G_f[v2][v1].cap = G_f[v2][v1].cap + cap   #actualizar arista original en la red residual
            else:                    # no es una arista de retroceso 
                if G_f[v1][v2].cost == -1: output+=1
                flow[v1][v2] += cap      #actualizar arista en la red original             
                G_f[v1][v2].cap = G_f[v1][v2].cap - cap     #actualizar arista original en la red residual
                G_f[v2][v1].cap = G_f[v2][v1].cap + cap      #actualizar arista de retroceso en la red residual
        path, cap = find_path(G_f, s, t)   
    return flow, output

def find_path(G:dict[int, dict[int, edge]], s:int, t:int):
    pi = Bellman_Ford(G, s)
    cap = inf
    path = []
    if not pi[t]==None:
        current = t
        while not current == s:
            previous = pi[current]
            cap = min(G[previous][current].cap,cap)
            path.append((previous,current))
            current=previous
    return path,cap

### Mejoras